<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#SparkSession" data-toc-modified-id="SparkSession-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>SparkSession</a></span></li><li><span><a href="#Sanity-checks" data-toc-modified-id="Sanity-checks-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Sanity checks</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Tables" data-toc-modified-id="Tables-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Tables</a></span><ul class="toc-item"><li><span><a href="#mrf" data-toc-modified-id="mrf-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>mrf</a></span></li><li><span><a href="#First-login" data-toc-modified-id="First-login-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>First login</a></span></li><li><span><a href="#Most-popular-category" data-toc-modified-id="Most-popular-category-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Most popular category</a></span></li></ul></li><li><span><a href="#Metrics" data-toc-modified-id="Metrics-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Metrics</a></span></li><li><span><a href="#Result" data-toc-modified-id="Result-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Result</a></span></li></ul></div>

In [1]:
import $ivy.`org.apache.spark::spark-core:3.1.2`
import $ivy.`org.apache.spark::spark-sql:3.1.2`
import $ivy.`org.apache.spark::spark-hive:3.1.2`
import $ivy.`org.apache.spark::spark-mllib:3.1.2`

import $ivy.$                                   

import $ivy.$                                  

import $ivy.$                                   

import $ivy.$                                    

https://spark.apache.org/docs/latest/sql-performance-tuning.html

# SparkSession

In [2]:
import org.apache.hadoop.fs.{FileSystem, Path, FSDataOutputStream}
import java.io.BufferedOutputStream

import org.apache.log4j.{Level, LogManager}
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.{col, row_number, max, count, from_unixtime, monotonicallyIncreasingId, to_date}
import org.apache.spark.sql.types.DateType

  
LogManager.getLogger("org").setLevel(Level.ERROR)
  
val spark = SparkSession.builder()
  .master("local[*]")
  .appName("myApp")
  .config("hive.metastore.uris","thrift://localhost:9083")
  .config("spark.hadoop.fs.default.name", "hdfs://localhost:9000")
  .enableHiveSupport()
  .getOrCreate()

import spark.implicits._ 

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import org.apache.hadoop.fs.{FileSystem, Path, FSDataOutputStream}

import java.io.BufferedOutputStream


import org.apache.log4j.{Level, LogManager}

import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.expressions.Window

import org.apache.spark.sql.functions.{col, row_number, max, count, from_unixtime, monotonicallyIncreasingId, to_date}

import org.apache.spark.sql.types.DateType

  

spark: SparkSession = org.apache.spark.sql.SparkSession@464c79b5
import spark.implicits._ 

# Sanity checks

In [3]:
spark.version

res2: String = "3.1.2"

In [4]:
spark.conf.get("spark.sql.autoBroadcastJoinThreshold")

res3: String = "10485760b"

In [5]:
spark.catalog.listDatabases().show(false)

21/09/28 14:53:18 INFO metastore: Trying to connect to metastore with URI thrift://localhost:9083
21/09/28 14:53:18 INFO metastore: Opened a connection to metastore, current connections: 1
21/09/28 14:53:18 INFO metastore: Connected to metastore.


+---------------+---------------------+------------------------------------------------------------+
|name           |description          |locationUri                                                 |
+---------------+---------------------+------------------------------------------------------------+
|default        |Default Hive database|hdfs://localhost:9000/user/hive/warehouse                   |
|hive_essentials|                     |hdfs://localhost:9000/user/hive/warehouse/hive_essentials.db|
|pluralsight    |                     |hdfs://localhost:9000/user/hive/warehouse/pluralsight.db    |
|rostel         |                     |hdfs://localhost:9000/user/hive/warehouse/rostel.db         |
|rostel_test    |                     |hdfs://localhost:9000/user/hive/warehouse/rostel_test.db    |
+---------------+---------------------+------------------------------------------------------------+



In [6]:
spark.sql("show tables in rostel").show()

+--------+----------------+-----------+
|database|       tableName|isTemporary|
+--------+----------------+-----------+
|  rostel|        bad_data|      false|
|  rostel|   first_session|      false|
|  rostel|        logs_mrf|      false|
|  rostel|popular_category|      false|
+--------+----------------+-----------+



# Load data

In [7]:
// app arg
val dt = "2019-12-11"
// app arg
val path = "hdfs:/user/sergey/clickstream"

val path_logs = path + "/" + dt.replaceAll("-","") + "/*.tsv"

dt: String = "2019-12-11"
path: String = "hdfs:/user/sergey/clickstream"
path_logs: String = "hdfs:/user/sergey/clickstream/20191211/*.tsv"

In [8]:
val schema_logs_raw = "timestamp string, url string, login string, region_code string"

val raw_logs = spark.
    read.
    options(Map("header"->"true", "sep"->"\t")).
    schema(schema_logs_raw).
    csv(path_logs).
    withColumn("id", monotonicallyIncreasingId).
    withColumn("valid_today", to_date(from_unixtime('timestamp)) === dt).
    withColumn("valid_not_today", to_date(from_unixtime('timestamp)) !== dt).
    withColumn("not_timestamp", from_unixtime('timestamp).isNull).cache()
raw_logs.printSchema

root
 |-- timestamp: string (nullable = true)
 |-- url: string (nullable = true)
 |-- login: string (nullable = true)
 |-- region_code: string (nullable = true)
 |-- id: long (nullable = false)
 |-- valid_today: boolean (nullable = true)
 |-- valid_not_today: boolean (nullable = true)
 |-- not_timestamp: boolean (nullable = false)



schema_logs_raw: String = "timestamp string, url string, login string, region_code string"
raw_logs: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [timestamp: string, url: string ... 6 more fields]

In [9]:
raw_logs.show(100)

+----------+--------------------+-----+-----------+----------+-----------+---------------+-------------+
| timestamp|                 url|login|region_code|        id|valid_today|valid_not_today|not_timestamp|
+----------+--------------------+-----+-----------+----------+-----------+---------------+-------------+
|1576022400|    http://yandex.ru|vasya|         77|         0|       true|          false|        false|
|1576022412|    http://google.ru|vasya|         77|         1|       true|          false|        false|
|       abc|http://shop.ru/go...|petya|         59|         2|       null|           null|         true|
|1576022500|    http://yandex.ru| vika|         59|         3|       true|          false|        false|
|1576022412|    http://google.ru|sveta|         77|         4|       true|          false|        false|
|1576022435|http://shop.ru/go...|kolya|         59|         5|       true|          false|        false|
|1576022500|http://shop.ru/go...| vika|        abc|    

In [10]:
val logs = raw_logs.
    filter('valid_today || 'not_timestamp).
    selectExpr(
        "cast(timestamp as long) timestamp", 
        "url", 
        "login", 
        "cast(region_code as integer) region_code",
        "id"
              )
logs.printSchema

root
 |-- timestamp: long (nullable = true)
 |-- url: string (nullable = true)
 |-- login: string (nullable = true)
 |-- region_code: integer (nullable = true)
 |-- id: long (nullable = false)



logs: org.apache.spark.sql.package.DataFrame = [timestamp: bigint, url: string ... 3 more fields]

In [11]:
logs.show(100,false)

+----------+----------------------------+-----+-----------+----------+
|timestamp |url                         |login|region_code|id        |
+----------+----------------------------+-----+-----------+----------+
|1576022400|http://yandex.ru            |vasya|77         |0         |
|1576022412|http://google.ru            |vasya|77         |1         |
|null      |http://shop.ru/goods/iphonex|petya|59         |2         |
|1576022500|http://yandex.ru            |vika |59         |3         |
|1576022412|http://google.ru            |sveta|77         |4         |
|1576022435|http://shop.ru/goods/iphonex|kolya|59         |5         |
|1576022500|http://shop.ru/goods/iphonex|vika |null       |6         |
|1576022700|http://google.ru            |vika |59         |7         |
|1576022412|http://yandex.ru            |sveta|77         |8         |
|1576022635|http://food-shop            |kolya|59         |9         |
|1576022900|http://sports-wiki.ru/apple/|kolya|1000       |10        |
|15760

In [12]:
val path_regions = "hdfs:/user/sergey/regions/regions.tsv"
val schema_regions = "regione_code integer, mrf string"
val regions = spark.
    read.
    options(Map("header"->"true", "sep"->"\t")).
    schema(schema_regions).
    csv(path_regions)

regions.printSchema

root
 |-- regione_code: integer (nullable = true)
 |-- mrf: string (nullable = true)



path_regions: String = "hdfs:/user/sergey/regions/regions.tsv"
schema_regions: String = "regione_code integer, mrf string"
regions: org.apache.spark.sql.package.DataFrame = [regione_code: int, mrf: string]

In [13]:
regions.show(100,false)

+------------+------+
|regione_code|mrf   |
+------------+------+
|45          |ural  |
|59          |ural  |
|31          |center|
|32          |center|
|77          |center|
+------------+------+



In [14]:
val path_categories = "hdfs:/user/sergey/categories/categories.tsv"
val schema_categories = "url string, category string"
val categories = spark.
    read.
    options(Map("header"->"true", "sep"->"\t")).
    schema(schema_categories).
    csv(path_categories)

categories.printSchema

root
 |-- url: string (nullable = true)
 |-- category: string (nullable = true)



path_categories: String = "hdfs:/user/sergey/categories/categories.tsv"
schema_categories: String = "url string, category string"
categories: org.apache.spark.sql.package.DataFrame = [url: string, category: string]

In [15]:
categories.show(100,false)

+----------------------------+--------+
|url                         |category|
+----------------------------+--------+
|http://yandex.ru            |search  |
|http://google.ru            |search  |
|http://shop.ru/goods/iphonex|shop    |
|http://food-shop.ru/apple/  |shop    |
|http://sports-wiki.ru/apple/|sport   |
+----------------------------+--------+



# Tables

## mrf

In [16]:
regions.show

+------------+------+
|regione_code|   mrf|
+------------+------+
|          45|  ural|
|          59|  ural|
|          31|center|
|          32|center|
|          77|center|
+------------+------+



In [17]:
val logs_mrf = logs.
    join(regions.hint("broadcast"), logs("region_code") === regions("regione_code"),"left").
    drop("regione_code")
logs_mrf.show(100, false)

+----------+----------------------------+-----+-----------+----------+------+
|timestamp |url                         |login|region_code|id        |mrf   |
+----------+----------------------------+-----+-----------+----------+------+
|1576022400|http://yandex.ru            |vasya|77         |0         |center|
|1576022412|http://google.ru            |vasya|77         |1         |center|
|null      |http://shop.ru/goods/iphonex|petya|59         |2         |ural  |
|1576022500|http://yandex.ru            |vika |59         |3         |ural  |
|1576022412|http://google.ru            |sveta|77         |4         |center|
|1576022435|http://shop.ru/goods/iphonex|kolya|59         |5         |ural  |
|1576022500|http://shop.ru/goods/iphonex|vika |null       |6         |null  |
|1576022700|http://google.ru            |vika |59         |7         |ural  |
|1576022412|http://yandex.ru            |sveta|77         |8         |center|
|1576022635|http://food-shop            |kolya|59         |9    

logs_mrf: org.apache.spark.sql.package.DataFrame = [timestamp: bigint, url: string ... 4 more fields]

In [18]:
// the join is broadcast
// logs_mrf.explain(true)
logs_mrf.explain

== Physical Plan ==
*(2) Project [timestamp#326L, url#52, login#53, region_code#327, id#59L, mrf#515]
+- *(2) BroadcastHashJoin [region_code#327], [regione_code#514], LeftOuter, BuildRight, false
   :- *(2) Project [cast(timestamp#51 as bigint) AS timestamp#326L, url#52, login#53, cast(region_code#54 as int) AS region_code#327, id#59L]
   :  +- *(2) Filter (valid_today#65 OR not_timestamp#80)
   :     +- InMemoryTableScan [id#59L, login#53, not_timestamp#80, region_code#54, timestamp#51, url#52, valid_today#65], [(valid_today#65 OR not_timestamp#80)]
   :           +- InMemoryRelation [timestamp#51, url#52, login#53, region_code#54, id#59L, valid_today#65, valid_not_today#72, not_timestamp#80], StorageLevel(disk, memory, deserialized, 1 replicas)
   :                 +- *(1) Project [timestamp#51, url#52, login#53, region_code#54, id#59L, (cast(from_unixtime(cast(timestamp#51 as bigint), yyyy-MM-dd HH:mm:ss, Some(Europe/Moscow)) as date) = 18241) AS valid_today#65, NOT (cast(from_unixt

In [19]:
spark.sql("use rostel")
logs_mrf.
    write.
    mode("overwrite").
    saveAsTable("logs_mrf")

21/09/28 14:53:27 INFO metastore: Mestastore configuration hive.metastore.filter.hook changed from org.apache.hadoop.hive.metastore.DefaultMetaStoreFilterHookImpl to org.apache.hadoop.hive.ql.security.authorization.plugin.AuthorizationMetaStoreFilterHook
21/09/28 14:53:27 INFO metastore: Closed a connection to metastore, current connections: 0
21/09/28 14:53:27 INFO metastore: Trying to connect to metastore with URI thrift://localhost:9083
21/09/28 14:53:27 INFO metastore: Opened a connection to metastore, current connections: 1
21/09/28 14:53:27 INFO metastore: Connected to metastore.
21/09/28 14:53:27 INFO metastore: Trying to connect to metastore with URI thrift://localhost:9083
21/09/28 14:53:27 INFO metastore: Opened a connection to metastore, current connections: 2
21/09/28 14:53:27 INFO metastore: Connected to metastore.


res18_0: org.apache.spark.sql.package.DataFrame = []

In [20]:
logs_mrf.rdd.partitions.size

res19: Int = 2

## First login

In [21]:
val w1 = Window.partitionBy('login).orderBy('timestamp.desc)
val first_session = logs.
    withColumn("rn", row_number.over(w1)).where('rn === 1).
    select('login,'timestamp)

first_session.show()

+-----+----------+
|login| timestamp|
+-----+----------+
| null|1576022412|
|sveta|1576022412|
|petya|1576022435|
|vasya|1576022435|
| vika|1576022700|
|kolya|1576022900|
+-----+----------+



w1: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@2d405825
first_session: org.apache.spark.sql.package.DataFrame = [login: string, timestamp: bigint]

In [22]:
// first_session.rdd.partitions.size

first_session.
    coalesce(2).
    write.
    mode("overwrite").
    saveAsTable("first_session")

## Most popular category

In [23]:
val logs_host = logs.selectExpr("login","from_unixtime(timestamp) as date", "parse_url(url, 'HOST') as host","url","id")
val categories_host = categories.selectExpr("parse_url(url, 'HOST') as host", "category")
val logs_category = logs_host.join(categories_host.hint("broadcast"), Seq("host"), "left")
logs_category.show(100, false)

+--------------+-----+-------------------+----------------------------+----------+--------+
|host          |login|date               |url                         |id        |category|
+--------------+-----+-------------------+----------------------------+----------+--------+
|yandex.ru     |vasya|2019-12-11 03:00:00|http://yandex.ru            |0         |search  |
|google.ru     |vasya|2019-12-11 03:00:12|http://google.ru            |1         |search  |
|shop.ru       |petya|null               |http://shop.ru/goods/iphonex|2         |shop    |
|yandex.ru     |vika |2019-12-11 03:01:40|http://yandex.ru            |3         |search  |
|google.ru     |sveta|2019-12-11 03:00:12|http://google.ru            |4         |search  |
|shop.ru       |kolya|2019-12-11 03:00:35|http://shop.ru/goods/iphonex|5         |shop    |
|shop.ru       |vika |2019-12-11 03:01:40|http://shop.ru/goods/iphonex|6         |shop    |
|google.ru     |vika |2019-12-11 03:05:00|http://google.ru            |7        

logs_host: org.apache.spark.sql.package.DataFrame = [login: string, date: string ... 3 more fields]
categories_host: org.apache.spark.sql.package.DataFrame = [host: string, category: string]
logs_category: org.apache.spark.sql.package.DataFrame = [host: string, login: string ... 4 more fields]

In [24]:
logs_category.groupBy('login,'category).count().show(100, false)

+-----+--------+-----+
|login|category|count|
+-----+--------+-----+
|petya|shop    |4    |
|vasya|shop    |4    |
|vika |search  |8    |
|sveta|null    |1    |
|vasya|search  |14   |
|kolya|sport   |4    |
|vika |shop    |4    |
|sveta|search  |7    |
|null |search  |1    |
|vasya|null    |1    |
|kolya|shop    |4    |
|kolya|null    |4    |
+-----+--------+-----+



In [25]:
val w2 = Window.partitionBy('login).orderBy('count.desc)
val popular_category = logs_category.
    groupBy('login,'category).count().
    withColumn("rn", row_number.over(w2)).
    filter('rn===1).
    drop('rn)

popular_category.show

+-----+--------+-----+
|login|category|count|
+-----+--------+-----+
| null|  search|    1|
|sveta|  search|    7|
|petya|    shop|    4|
|vasya|  search|   14|
| vika|  search|    8|
|kolya|   sport|    4|
+-----+--------+-----+



w2: org.apache.spark.sql.expressions.WindowSpec = org.apache.spark.sql.expressions.WindowSpec@3a10ec9d
popular_category: org.apache.spark.sql.package.DataFrame = [login: string, category: string ... 1 more field]

In [26]:
// popular_category.rdd.partitions.size

popular_category.
    coalesce(2).
    write.
    mode("overwrite").
    saveAsTable("popular_category")

# Metrics

In [27]:
// Total
val num_total = raw_logs.count

// Today timestamps
val num_timestamp_today = raw_logs.filter('valid_today).count

//  Not today
val id_timestamp_not_today = raw_logs.filter('valid_not_today).select("id").as[Long].collect
val num_timestamps_not_today = id_timestamp_not_today.size

// Not valid timestamp
val id_not_timestamp = raw_logs.filter('not_timestamp).select("id").as[Long].collect
val num_not_timestamps = id_not_timestamp.size

// Empty login
val id_empty_login = raw_logs.filter('login.isNull).select("id").as[Long].collect
val num_empty_login = id_empty_login.size

// "Bad" URL: the host is not in the lookup file
val id_bad_url = logs_category.
    filter('category.isNull).
    select("id").as[Long].collect
val num_bad_url = id_bad_url.size

// "Bad" region
val id_bad_region = logs.
    filter(('region_code.isNull) || ('region_code > 100) || ('region_code < 0 )).
    select("id").
    as[Long].
    collect
val num_bad_region = id_bad_region.size

val path_metrics = path + "/" + dt.replaceAll("-","") + "/" + "metrics.txt"
println(path_metrics)
val fs = FileSystem.get(spark.sparkContext.hadoopConfiguration)
val output = fs.create(new Path(path_metrics))
val os = new java.io.BufferedOutputStream(output)
os.write(s"""
      Total number of records is : $num_total
Number of valid today timestamps : $num_timestamp_today
  Number of not valid timestamps : $num_not_timestamps
          Number of empty logins : $num_empty_login
              Number of bad URLs : $num_bad_url
       Number of bad region_code : $num_bad_region  
""".getBytes("UTF-8"))
os.close()


// Bad logs table
val bad_logs_id = (id_timestamp_not_today ++ id_not_timestamp ++ id_empty_login ++ id_bad_url ++ id_bad_region)
val bad_logs = raw_logs.filter('id.isin(bad_logs_id:_*)).select("timestamp","url","login","region_code","id")
bad_logs.
    write.
    mode("overwrite").
    saveAsTable("bad_data")
bad_logs.show

hdfs:/user/sergey/clickstream/20191211/metrics.txt
+----------+--------------------+-----+-----------+----------+
| timestamp|                 url|login|region_code|        id|
+----------+--------------------+-----+-----------+----------+
|       abc|http://shop.ru/go...|petya|         59|         2|
|1576022500|http://shop.ru/go...| vika|        abc|         6|
|1576022635|    http://food-shop|kolya|         59|         9|
|1576022900|http://sports-wik...|kolya|       1000|        10|
|1576022400|                 123|vasya|         77|        14|
|      null|    http://google.ru|sveta|         77|        18|
|1576022412|                null|sveta|         77|        22|
|1576022635|    http://food-shop|kolya|         59|        23|
|1576022412|    http://google.ru| null|         77|        26|
|1576022435|http://shop.ru/go...|petya|       null|        30|
|1576022635|    http://food-shop|kolya|         59|        37|
|1576022635|    http://food-shop|kolya|         59|8589934601|
+---

num_total: Long = 56L
num_timestamp_today: Long = 54L
id_timestamp_not_today: Array[Long] = Array()
num_timestamps_not_today: Int = 0
id_not_timestamp: Array[Long] = Array(2L, 18L)
num_not_timestamps: Int = 2
id_empty_login: Array[Long] = Array(26L)
num_empty_login: Int = 1
id_bad_url: Array[Long] = Array(9L, 14L, 22L, 23L, 37L, 8589934601L)
num_bad_url: Int = 6
id_bad_region: Array[Long] = Array(6L, 10L, 30L)
num_bad_region: Int = 3
path_metrics: String = "hdfs:/user/sergey/clickstream/20191211/metrics.txt"
fs: FileSystem = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_1007489717_24, ugi=sergey (auth:SIMPLE)]]
output: FSDataOutputStream = FSDataOutputStream{wrappedStream=DFSOutputStream:blk_1073742359_1535}
os: BufferedOutputStream = java.io.BufferedOutputStream@4fbba927
bad_logs_id: Array[Long] = Array(
  2L,
  18L,
  26L,
  9L,
  14L,
  22L,
  23L,
  37L,
  8589934601L,
  6L,
  10L,
  30L
)
bad_logs: org.apache.spark.sql.package.DataFrame = [timestamp: string, url: string ... 3 mo

# Result

In [28]:
spark.sql("show tables in rostel").show()

+--------+----------------+-----------+
|database|       tableName|isTemporary|
+--------+----------------+-----------+
|  rostel|        bad_data|      false|
|  rostel|   first_session|      false|
|  rostel|        logs_mrf|      false|
|  rostel|popular_category|      false|
+--------+----------------+-----------+

